In [1]:
from input_timeout import input_to
from imp import reload
from keras import backend as K
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
import vgg16x
import math

Using Theano backend.


WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29



ERROR (theano.sandbox.cuda): ERROR: Not using GPU. Initialisation of device gpu failed:
initCnmem: cnmemInit call failed! Reason=CNMEM_STATUS_OUT_OF_MEMORY. numdev=1



RuntimeError: Cuda error: kernel_reduce_ccontig_node_m23eb44cb0bd6afa880a56f8b1f6a7d7f_0: out of memory. (grid: 1 x 1; block: 256 x 1 x 1)

Apply node that caused the error: GpuCAReduce{add}{1}(<CudaNdarrayType(float32, vector)>)
Toposort index: 0
Inputs types: [CudaNdarrayType(float32, vector)]
Inputs shapes: [(10000,)]
Inputs strides: [(1,)]
Inputs values: ['not shown']
Outputs clients: [[HostFromGpu(GpuCAReduce{add}{1}.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

In [7]:
from utils import load_array,save_array

AttributeError: module 'theano' has no attribute 'tests'

In [5]:
K.set_image_dim_ordering('th')
reload(vgg16x);reload(utils)

NameError: name 'K' is not defined

In [6]:
id0 = utils.load_array('trnLabels.bc')

NameError: name 'utils' is not defined

In [3]:
model = utils.vgg_ft(2)
model_path = './results_redux/'
model_name = 'ft7_17-08-02_22:50:23.h5'
model.load_weights(model_path + model_name)

(None, 3, 224, 224)


(None, 64, 112, 112)
(None, 128, 56, 56)


(None, 256, 28, 28)


(None, 512, 14, 14)


(None, 512, 7, 7)


vgg16 is initialized.


OSError: Unable to open file (Unable to open file: name = './results_redux/ft7_17-08-02_22:50:23.h5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

In [4]:
layers = model.layers
last_conv_idx = [index for index,layer in enumerate(layers) 
                     if type(layer) is Conv2D][-1]

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 64, 114, 114)      0         
__________

conv2d_7 (Conv2D)            (None, 256, 56, 56)       590080    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 256, 28, 28)       0         
_________________________________________________________________
zero_padding2d_8 (ZeroPaddin (None, 256, 30, 30)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 512, 28, 28)       1180160   
_________________________________________________________________
zero_padding2d_9 (ZeroPaddin (None, 512, 30, 30)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 512, 28, 28)       2359808   
_________________________________________________________________
zero_padding2d_10 (ZeroPaddi (None, 512, 30, 30)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 512, 28, 28)       2359808   
__________

conv2d_13 (Conv2D)           (None, 512, 14, 14)       2359808   
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 512, 7, 7)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              102764544 
_________________________________________________________________
dropout_1 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dropout_2 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 8194      


Total params: 134,268,738
Trainable params: 8,194
Non-trainable params: 134,260,544
_________________________________________________________________


In [6]:
conv_layers = layers[:last_conv_idx+1]
conv_model = Sequential(conv_layers)
fc_layers = layers[last_conv_idx+1:]

In [8]:
batch_size = 64
batches = utils.get_batches('./train_redux', shuffle=False, batch_size=batch_size)
val_batches = utils.get_batches('./valid_redux', shuffle=False, batch_size=batch_size)
test_batches = utils.get_batches('./test_redux', shuffle=False, batch_size=batch_size, class_mode=None)

val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = utils.onehot(val_classes)
trn_labels = utils.onehot(trn_classes)

Found 23000 images belonging to 2 classes.


Found 2000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [11]:
import numpy as np
filenames = test_batches.filenames
ids = np.array([int(f[8:f.find('.')]) for f in filenames])
ids.shape

(12500,)

In [7]:
utils.save_array('test_id.bc', ids)
ids_l = utils.load_array('test_id.bc')

NameError: name 'ids' is not defined

In [33]:
id0 = utils.load_array('trnLabels_0.bc')
id1 = utils.load_array('trnLabels_2.bc')

In [34]:
x = (id0 == id1)
print(x[1,0])
for i in range(10):
    if x[i,1] != True:
        print('xx')

True


In [19]:
print(ids[:20])

[ 4425  4207  9577  7612  8312  1761 10976  2375 10699 11315  6864  1757 12288     2   407  4432
  5152   288  9578  7720]


In [7]:
trn_features = conv_model.predict_generator(batches, math.ceil(batches.samples / batch_size), verbose=2)
val_features = conv_model.predict_generator(val_batches, math.ceil(val_batches.samples / batch_size), verbose=2)
tst_features = conv_model.predict_generator(test_batches, math.ceil(test_batches.samples / batch_size), verbose=2)

In [8]:
print(trn_features.shape)
print(val_features.shape)
print(tst_features.shape)

(23000, 512, 14, 14)
(2000, 512, 14, 14)
(12500, 512, 14, 14)


In [9]:
utils.save_array('train_convlayer_features.bc', trn_features)
utils.save_array('valid_convlayer_features.bc', val_features)
utils.save_array('test_convlayer_features.bc', tst_features)